In [10]:

%ls
%cd akbank-genai-chatbot/
%ls
!mv /content/GeminiAPI.ipynb .
%ls

data/  README.md
[Errno 2] No such file or directory: 'akbank-genai-chatbot/'
/content/akbank-genai-chatbot
data/  README.md
mv: cannot stat '/content/GeminiAPI.ipynb': No such file or directory
data/  README.md


In [1]:
!git clone https://github.com/gizemnuryilmaz/akbank-genai-chatbot.git

Cloning into 'akbank-genai-chatbot'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (14/14), done.
Receiving objects: 100% (15/15), 10.34 KiB | 10.34 MiB/s, done.
Resolving deltas: 100% (1/1), done.
remote: Total 15 (delta 1), reused 10 (delta 0), pack-reused 0 (from 0)


In [2]:
!pip install langchain_google_genai --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.2/467.2 kB 18.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.3.27 requires langchain-core<1.0.0,>=0.3.72, but you have langchain-core 1.0.0 which is incompatible.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.8.0 which is incompatible.


In [6]:
from langchain_google_genai import ChatGoogleGenerativeAI
from google.colab import userdata
import os
os.environ["GOOGLE_API_KEY"]=userdata.get("GOOGLE_API_KEY")
gemini=ChatGoogleGenerativeAI(
    model="gemini-2.0-flash"
)
#response=gemini.invoke("Kahve nedir?")
#print(response.content)


In [4]:
!pip install -qU "langchain[google-genai]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.4/155.4 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.2/106.2 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.6/207.6 kB 14.3 MB/s eta 0:00:00


In [5]:
import os
from google import genai
from google.genai import types
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# 1. API Anahtarını Ayarlama
# Ortam değişkeninden anahtarı alacaktır.
try:
    client = genai.Client()
except Exception as e:
    print(f"Hata: Gemini API anahtarı ayarlanmamış veya geçersiz. Lütfen 'export GEMINI_API_KEY=\"...\"' komutunu kullanın. Detay: {e}")
    exit()

# 2. Veri Seti (Knowledge Base) Hazırlama
# Bu basit örnekte, veri setimiz bir Python listesi içindeki metin parçalarıdır.
KNOWLEDGE_BASE = [
    "Gemini 2.0 Flash, Google tarafından geliştirilmiş, hızlı ve çok modlu bir büyük dil modelidir (LLM).",
    "RAG (Retrieval-Augmented Generation), harici bir bilgi tabanından alakalı verileri alıp, bu verileri LLM'e bağlam olarak sağlayarak yanıt üretimini geliştiren bir tekniktir.",
    "Gemini modelleri, hem metin hem de görsel/işitsel verileri işleyebilir ve yanıtlayabilir.",
    "Gemini 2.0 Flash'ın temel avantajları arasında düşük gecikme süresi, yüksek hız ve multimodal yetenekler bulunur.",
    "Bir RAG sisteminde, veriler genellikle 'embedding' adı verilen sayısal vektörlere dönüştürülür.",
    "En iyi uygulama, verileri 200 ile 500 kelimelik küçük ve bağlamsal parçalara (chunk) ayırmaktır.",
]

# Gömme (Embedding) modelini seçme
EMBEDDING_MODEL = 'gemini-embedding-001' # Gemini için önerilen embedding modeli

def create_embeddings(texts):
    """Metin listesi için gömme (embedding) vektörlerini oluşturur."""
    print("Veri setinin embedding'leri oluşturuluyor...")
    response = client.models.embed_content(
        model=EMBEDDING_MODEL,
        contents=texts,
    )
    embeddings_list = [item.values for item in response.embeddings]
    # Vektörleri numpy dizisine dönüştür
    return np.array(embeddings_list)

# 3. Vektör Veritabanı Oluşturma (Bu örnekte bellek içi bir matris)
knowledge_base_embeddings = create_embeddings(KNOWLEDGE_BASE)
print(f"Oluşturulan embedding sayısı: {len(knowledge_base_embeddings)}")

def retrieve_context(query, top_k=2):
    """Sorgu ile en alakalı veri parçalarını bulur (Geri Alma)."""
    # Sorgunun embedding'ini oluştur
    query_embedding_response = client.models.embed_content(
        model=EMBEDDING_MODEL,
        contents=[query]
    )
    query_embeddings_list = [item.values for item in query_embedding_response.embeddings]
    query_embedding = np.array(query_embeddings_list)

    # Kosinüs benzerliği ile sorgu ve bilgi tabanı vektörleri arasındaki benzerliği hesapla
    # reshape(1, -1) sorgu vektörünün tek satırlı bir matris olmasını sağlar
    similarities = cosine_similarity(query_embedding.reshape(1, -1), knowledge_base_embeddings)

    # En yüksek benzerliğe sahip parçaların indekslerini al
    top_indices = np.argsort(similarities[0])[::-1][:top_k]

    # En alakalı parçaları döndür
    retrieved_chunks = [KNOWLEDGE_BASE[i] for i in top_indices]
    print(f"\n[Geri Alınan Bağlam ({top_k} adet)]:")
    for chunk in retrieved_chunks:
        print(f"- {chunk}")

    return "\n".join(retrieved_chunks)

def generate_response(query, context):
    """Geri alınan bağlamı kullanarak Gemini'den yanıt üretir (Üretim)."""

    # Modelden istediğimiz davranışı belirten prompt şablonu
    prompt = f"""
    Aşağıdaki 'BAĞLAM' bölümünde sağlanan bilgilere dayanarak '{query}' sorusuna yanıt verin.
    Eğer bağlamda yeterli bilgi yoksa, 'Üzgünüm, bu bilgiye sahip değilim.' diye cevap verin.

    BAĞLAM:
    {context}

    SORU: {query}
    YANIT:
    """

    print("\n[Yanıt Üretiliyor...]")
    response = client.models.generate_content(
        model='gemini-2.0-flash', # Hızlı ve verimli model
        contents=prompt
    )

    return response.text

def simple_rag_pipeline(query):
    """Tüm RAG sürecini çalıştırır."""
    # 4. Geri Alma (Retrieval)
    context = retrieve_context(query)

    # 5. Üretim (Generation)
    final_response = generate_response(query, context)

    print("\n--- Nihai Yanıt ---")
    print(final_response)
    print("--------------------")

# Örnek Sorgu
query1 = "Gemini 2.0 Flash'ın ana faydaları nelerdir?"
simple_rag_pipeline(query1)

print("\n" + "="*50 + "\n")

# Bağlamda olmayan bir sorgu
query2 = "Fransa'nın başkenti neresidir?"
simple_rag_pipeline(query2)

Veri setinin embedding'leri oluşturuluyor...
Oluşturulan embedding sayısı: 6

[Geri Alınan Bağlam (2 adet)]:
- Gemini 2.0 Flash'ın temel avantajları arasında düşük gecikme süresi, yüksek hız ve multimodal yetenekler bulunur.
- Gemini 2.0 Flash, Google tarafından geliştirilmiş, hızlı ve çok modlu bir büyük dil modelidir (LLM).

[Yanıt Üretiliyor...]

--- Nihai Yanıt ---
Gemini 2.0 Flash'ın ana faydaları düşük gecikme süresi, yüksek hız ve multimodal yetenekleridir.

--------------------



[Geri Alınan Bağlam (2 adet)]:
- RAG (Retrieval-Augmented Generation), harici bir bilgi tabanından alakalı verileri alıp, bu verileri LLM'e bağlam olarak sağlayarak yanıt üretimini geliştiren bir tekniktir.
- Gemini 2.0 Flash, Google tarafından geliştirilmiş, hızlı ve çok modlu bir büyük dil modelidir (LLM).

[Yanıt Üretiliyor...]

--- Nihai Yanıt ---
Üzgünüm, bu bilgiye sahip değilim.

--------------------


In [ ]:
from google.colab import drive
drive.mount('/content/drive')